In [1]:
import pandas as pd
from difflib import get_close_matches
import redis

# Load the dataset
file_path = r"D:\OneDrive\Desktop\new\dataset\Drug prescription to disease\final.csv"
df = pd.read_csv(file_path)
df.dropna(inplace=True)
df["disease"] = df["disease"].str.lower().str.strip()
df["drug"] = df["drug"].str.lower().str.strip()

# Connect to Redis for drug interaction checking
redis_conn = redis.Redis()

def get_disease_from_symptom(symptom_query, top_n=3):
    """Get closest matching diseases for a symptom query"""
    symptom_query = symptom_query.lower().strip()
    all_diseases = df["disease"].unique()
    matches = get_close_matches(symptom_query, all_diseases, n=top_n, cutoff=0.5)
    return matches

def recommend_drugs(disease_name):
    """Return drug(s) recommended for a disease"""
    disease_name = disease_name.lower().strip()
    recommended = df[df["disease"] == disease_name]["drug"].unique().tolist()
    return recommended

def check_interaction(drug1, drug2):
    key = f"{drug1.lower()}:{drug2.lower()}"
    result = redis_conn.get(key)
    if result:
        return result.decode("utf-8")
    else:
        return "No known interaction found."


In [2]:

# Example usage:
symptoms = "cold"
matched_diseases = get_disease_from_symptom(symptoms)
for disease in matched_diseases:
    print("Disease:", disease)
    drugs = recommend_drugs(disease)
    print("Drugs:", drugs)
    for i in range(len(drugs)):
        for j in range(i + 1, len(drugs)):
            interaction = check_interaction(drugs[i], drugs[j])
            print(f"Interaction between {drugs[i]} and {drugs[j]}: {interaction}")


Disease: copd
Drugs: ['prednisone', 'montelukast', 'budesonide / formoterol', 'fluticasone / vilanterol', 'aclidinium', 'levalbuterol', 'roflumilast', 'umeclidinium / vilanterol', 'olodaterol / tiotropium', 'dyphylline', 'fluticasone / umeclidinium / vilanterol', 'olodaterol', 'umeclidinium']


ConnectionError: Error 10061 connecting to localhost:6379. No connection could be made because the target machine actively refused it.